# Create boundary conditions for Pb

In [1]:
import netCDF4 as nc
import numpy as np
import datetime
import csv
import pandas as pd
import pickle
from scipy.signal import savgol_filter
import xarray as xr
import scipy as sc

%matplotlib inline

#### Parameters

In [2]:
# domain dimensions:
imin, imax = 1139, 2179
jmin, jmax = 159, 799

# Rimwidths:
rimwidthN  = 10
rimwidthS  = 10
rimwidthW  = 10
rimwidthE  = 20

# Boundary coordinates: (i1,i2,j1,j2)
bdy_NCB = (1600,imax,jmax,jmax) # Northern Canada Basin
bdy_WCB = (imax,imax,391,jmax)  # Western Canada Basin
bdy_BB  = (imin,imin,449,674)   # Baffin Bay
bdy_HB  = (1189,1474,291,291)   # Hudson Bay

#### Files

In [20]:
mesh       = nc.Dataset('/ocean/brogalla/GEOTRACES/data/ANHA12/ANHA12_mesh1.nc')
mesh_lon   = np.array(mesh.variables['nav_lon'])
mesh_lat   = np.array(mesh.variables['nav_lat'])
mesh_bathy = np.array(mesh.variables['tmask'][0,:,:,:])

bathy_masked = np.ma.masked_where((mesh_bathy > 0.1), mesh_bathy)
ocean_masked = np.ma.masked_where((mesh_bathy < 0.1), mesh_bathy)

In [21]:
m      = nc.Dataset('/data/brogalla/run_storage/Mn-reference-202110/ANHA12_ref-2002_20211012/'+\
                    'ANHA12_EXH006_2002_monthly.nc', 'r')
mdepth = np.array(m.variables['deptht'])
mlons  = np.array(m.variables['nav_lon'])
mlats  = np.array(m.variables['nav_lat'])

### Make fields

##### Western CB boundary (model 'north')

In [22]:
ANHA12lat_CB = mesh_lat[bdy_WCB[0],bdy_WCB[2]:bdy_WCB[3]]
Pb_CB        = np.empty_like(mesh_bathy[:,bdy_WCB[0],bdy_WCB[2]:bdy_WCB[3]])
yy_CB, zz_CB = np.meshgrid(ANHA12lat_CB, mdepth)

In [23]:
Pb_CB.fill(2) 
Pb_CB[zz_CB <= 50]                        = 5 
Pb_CB[(zz_CB < 100) & (zz_CB > 50)]       = 4 
Pb_CB[(zz_CB < 100) & (zz_CB > 50) & (yy_CB < 72.5)] = 8  # coastal high band
Pb_CB[(zz_CB <= 300) & (zz_CB > 100)]     = 3
Pb_CB[(zz_CB > 300)]                      = 1.5

In [24]:
# smoothed_Pb_CB1 = np.empty_like(Pb_CB)
# smoothed_Pb_CB = np.empty_like(Pb_CB)

# # Smooth vertically (along depth)
# for i in range(0, len(Pb_CB[0,:])):
#     smoothed_Pb_CB[:,i] = savgol_filter(Pb_CB[:,i], 5, 3)

# # Smooth horizontally along cross section
# for depth in range(0, len(Pb_CB[:,0])):
#     smoothed_Pb_CB[depth,:] = savgol_filter(smoothed_Pb_CB1[depth,:], 51, 3)

smoothed_Pb_CB = sc.ndimage.filters.gaussian_filter(Pb_CB, 1.8, order=0)

Variable to save to file:

In [25]:
dPb_North          = np.empty((1,50,1,yy_CB.shape[1]*rimwidthN))
dPb_North[0,:,0,:] = np.tile(smoothed_Pb_CB, rimwidthN)

In [26]:
np.count_nonzero(dPb_North==0)

0

In [32]:
print('Model northern (west CB) boundary condition shape: ', dPb_North.shape)

Model northern (west CB) boundary condition shape:  (1, 50, 1, 4080)


##### Northern Canada Basin (model 'east' boundary)

In [51]:
ANHA12lon_AO = mesh_lon[bdy_NCB[0]:bdy_NCB[1],bdy_NCB[2]]
Pb_AO        = np.empty_like(mesh_bathy[:,bdy_NCB[0]:bdy_NCB[1],bdy_NCB[2]])
xx_AO, zz_AO = np.meshgrid(ANHA12lon_AO, mdepth)

In [52]:
# Pb_AO.fill(3) 
# Pb_AO[zz_AO < 50]                     = 5 
# Pb_AO[(zz_AO > 50)  & (zz_AO < 100)]  = 8 
# Pb_AO[(zz_AO > 100) & (zz_AO < 400)]  = 6 
# Pb_AO[(zz_AO > 400) & (zz_AO < 800)]  = 3 
# Pb_AO[(zz_AO > 800) & (zz_AO < 2000)] = 4
# Pb_AO[zz_AO > 2000]                   = 3

In [53]:
# Pb_AO.fill(3) 
# Pb_AO[zz_AO <= 50]                    = 5 
# Pb_AO[(zz_AO < 500) & (zz_AO > 50)]   = 3
# # Pb_AO[(zz_AO > 50) & (xx_AO < -160)] = 2 

Pb_AO.fill(3) 
Pb_AO[zz_AO <= 50]                    = 5 
Pb_AO[(zz_AO < 100) & (zz_AO > 50)]   = 4 
Pb_AO[(zz_AO > 300)]                  = 1.5
# Pb_AO[(zz_AO < 500) & (zz_AO > 50)]   = 3

In [54]:
# smoothed_Pb_AO1 = np.empty_like(Pb_AO)
# smoothed_Pb_AO = np.empty_like(Pb_AO)

# # Smooth vertically (along depth)
# for i in range(0, len(Pb_AO[0,:])):
#     smoothed_Pb_AO1[:,i] = savgol_filter(Pb_AO[:,i], 13, 3, mode='nearest')

# # Smooth horizontally along cross section
# for depth in range(0, len(Pb_AO[:,0])):
#     smoothed_Pb_AO[depth,:] = savgol_filter(smoothed_Pb_AO1[depth,:], 53, 3, mode='mirror')
    
# smoothed_Pb_AO = sc.ndimage.filters.gaussian_filter(smoothed_Pb_AO, 1.8, order=0, mode='wrap')
smoothed_Pb_AO = sc.ndimage.filters.gaussian_filter(Pb_AO, 1.8, order=0)

In [55]:
dPb_East   = np.empty((1,50,1,xx_AO.shape[1]*rimwidthE))
dPb_East[0,:,0,:]  = np.tile(smoothed_Pb_AO, rimwidthE)

print('Northern CB (model east) boundary condition shape: ', dPb_East.shape)

Northern CB (model east) boundary condition shape:  (1, 50, 1, 11580)


##### Southern Baffin Bay boundary

In [56]:
ANHA12lon_BB = mesh_lon[bdy_BB[0],bdy_BB[2]:bdy_BB[3]]
xx, zz = np.meshgrid(ANHA12lon_BB, mdepth) 

In [57]:
Pb_BB = np.empty_like(mesh_bathy[:,bdy_BB[0],bdy_BB[2]:bdy_BB[3]])

Pb_BB.fill(20)
Pb_BB[zz < 20]                               = 12
Pb_BB[(zz < 20)  & (xx > -50)]               = 16
Pb_BB[(zz < 100) & (zz > 20)]                = 18
Pb_BB[(zz < 100) & (zz > 20) & (xx < -50)]   = 14 
Pb_BB[(zz > 350) & (zz < 1500) & (xx < -41)] = 33 
Pb_BB[(zz > 350) & (zz < 1500) & (xx < -52)] = 23 
Pb_BB[zz > 2500]                             = 10

In [58]:
smoothed_Pb_BB1 = np.empty_like(Pb_BB)
smoothed_Pb_BB = np.empty_like(Pb_BB)

# Smooth vertically (along depth)
for i in range(0, len(Pb_BB[0,:])):
    smoothed_Pb_BB1[:,i] = savgol_filter(Pb_BB[:,i], 21, 3)

# Smooth horizontally along cross section
for depth in range(0, len(Pb_BB[:,0])):
    smoothed_Pb_BB[depth,:] = savgol_filter(smoothed_Pb_BB1[depth,:], 81, 3)

In [59]:
filtered_Pb_BB = sc.ndimage.filters.gaussian_filter(smoothed_Pb_BB, 1.8, order=0, mode='wrap')

In [60]:
dPb_South          = np.empty((1,50,1,xx.shape[1]*rimwidthS))
dPb_South[0,:,0,:] = np.tile(filtered_Pb_BB, rimwidthS) 

In [61]:
print('Southern boundary condition shape: ', dPb_South.shape)

Southern boundary condition shape:  (1, 50, 1, 2250)


##### Western Hudson Bay boundary

In [62]:
ANHA12lon_HB = mesh_lon[bdy_HB[0]:bdy_HB[1],bdy_HB[2]] 
xx_HB, zz_HB = np.meshgrid(ANHA12lon_HB, mdepth) 

In [63]:
Pb_HB = np.empty_like(mesh_bathy[:,bdy_HB[0]:bdy_HB[1],bdy_HB[2]])

Pb_HB.fill(12)
# Pb_BB[zz < 20]                               = 12
# Pb_BB[(zz < 20)  & (xx > -50)]               = 16
# Pb_BB[(zz < 100) & (zz > 20)]                = 18
# Pb_BB[(zz < 100) & (zz > 20) & (xx < -50)]   = 14 
# Pb_BB[(zz > 350) & (zz < 1500)]              = 33
# Pb_BB[(zz > 350) & (zz < 1500) & (xx < -41)] = 23 
# Pb_BB[(zz > 350) & (zz < 1500) & (xx > -36)] = 23 
# Pb_BB[zz > 2500]                             = 10

In [64]:
# smoothed_Pb_HB1 = np.empty_like(Pb_HB)
# smoothed_Pb_HB = np.empty_like(Pb_HB)

# # Smooth vertically (along depth)
# for i in range(0, len(Pb_HB[0,:])):
#     smoothed_Pb_HB1[:,i] = savgol_filter(Pb_HB[:,i], 21, 3)

# # Smooth horizontally along cross section
# for depth in range(0, len(Pb_HB[:,0])):
#     smoothed_Pb_HB[depth,:] = savgol_filter(smoothed_Pb_HB1[depth,:], 81, 3)

In [65]:
dPb_West          = np.empty((1,50,1,xx_HB.shape[1]*rimwidthW))
dPb_West[0,:,0,:] = np.tile(Pb_HB, rimwidthW) 

In [66]:
print('Western Hudson Bay boundary condition shape: ', dPb_West.shape)

Western Hudson Bay boundary condition shape:  (1, 50, 1, 2850)


#### Write NetCDF BC files:

In [67]:
# Check whether any of the values are zero
print(np.count_nonzero(dPb_South==0), np.count_nonzero(dPb_East==0), \
      np.count_nonzero(dPb_North==0), np.count_nonzero(dPb_West==0))

0 0 0 0


In [69]:
file_write = xr.Dataset(
    {'dPb_N' : (("time_counter","deptht","y","x1"), dPb_North*10**(-12)), \
     'dPb_S' : (("time_counter","deptht","y","x2"), dPb_South*10**(-12)), \
     'dPb_E' : (("time_counter","deptht","y","x3"), dPb_East*10**(-12)), \
     'dPb_W' : (("time_counter","deptht","y","x4"), dPb_West*10**(-12))}, 
    coords = {
        "time_counter": np.zeros(1),
        "deptht": np.zeros(50),
        "y": np.zeros(1),
        "x1": np.zeros(yy_CB.shape[1]*rimwidthN),
        "x2": np.zeros(xx.shape[1]*rimwidthS),
        "x3": np.zeros(xx_AO.shape[1]*rimwidthE),
        "x4": np.zeros(xx_HB.shape[1]*rimwidthW),
    },
)

# file_write.to_netcdf('/ocean/brogalla/GEOTRACES/data/Pb_OBC_20220322.nc', unlimited_dims='time_counter')